In [163]:
import pandas as pd
import numpy as np
import pandas_ta as ta
import plotly.graph_objects as go
import plotly.express as px
import yfinance as yf

In [164]:
data =  pd.read_csv('nifty_200/5M/ABCAPITAL.csv')

In [165]:
data['Datetime'] = pd.to_datetime(data['Datetime'])
data.set_index('Datetime',inplace=True)

In [166]:

def calculate_macd(data, fast, slow, signal):
    data['MACD_F'] = data.ta.macd(fast=fast, slow=slow, signal=signal)[f'MACD_{fast}_{slow}_{signal}']
    data['MACD_S'] = data.ta.macd(fast=fast, slow=slow, signal=signal)[f'MACDs_{fast}_{slow}_{signal}']
    data['MACD_H'] = data.ta.macd(fast=fast, slow=slow, signal=signal)[f'MACDh_{fast}_{slow}_{signal}']
    return data


In [167]:
def custom_resample(df, start='09:15', end='15:30', freq='1f'):
    df = df.between_time(start, end)
    return df.resample(freq, offset='0h15min').agg(
        {
            'Open': 'first',
            'High': 'max',
            'Low': 'min',
            'Close': 'last',
            'Volume': 'sum',
        }
    )


In [168]:
def calculate_pivot_levels(high, low, close):
    pp = (high + low + close) / 3
    tc = (pp + high) / 2
    bc = (pp + low) / 2
    r1 = 2 * pp - low
    r2 = pp + (high - low)
    r3 = high + 2 * (pp - low)
    s1 = 2 * pp - high
    s2 = pp - (high - low)
    s3 = low - 2 * (high - pp)
    
    return pp, tc, bc, r1, r2, r3, s1, s2, s3

In [169]:
def apply_pivots(df):
    # Shift OHLC columns to get yesterday's data
    df['Prev_High'] = df['High'].shift(1)
    df['Prev_Low'] = df['Low'].shift(1)
    df['Prev_Close'] = df['Close'].shift(1)

    # Apply the function to calculate pivot levels using yesterday's data
    df[['PP', 'TC', 'BC', 'R1', 'R2', 'R3', 'S1', 'S2', 'S3']] = df.apply(
        lambda row: calculate_pivot_levels(row['Prev_High'], row['Prev_Low'], row['Prev_Close']), 
        axis=1, 
        result_type='expand'
    )
    return df


In [170]:
df_daily = custom_resample(data,freq='D').dropna()

In [171]:
df_daily = apply_pivots(df_daily)

In [172]:
data['Date'] = pd.to_datetime(data.index).date

In [173]:
data['VWMA_10'] = ta.vwma(data['Close'],volume=data['Volume'], length=10)
data['Supertrend'] = data.ta.supertrend(length=10, multiplier=4)['SUPERT_10_4.0']


In [174]:
df_daily['Date'] =  df_daily.index.date

In [175]:
data.reset_index(inplace=True)

In [176]:
df_daily[pd.isna(df_daily['PP'])]

,Open,High,Low,Close,Volume,Prev_High,Prev_Low,Prev_Close,PP,TC,BC,R1,R2,R3,S1,S2,S3,Date
Datetime,,,,,,,,,,,,,,,,,,
2020-07-03 00:15:00,67.85,68.0,65.1,65.6,3021321,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-07-03


In [177]:
df_daily['%CPR'] = abs(df_daily['TC'] - df_daily['BC'])/df_daily['PP']

In [178]:
df_daily['%CPR'].mean()

0.017129838053762395

In [179]:
fully_merged   =  pd.merge(data.dropna(),df_daily[[ 'Date','PP', 'TC', 'BC', 'R1', 'R2', 'R3', 'S1', 'S2', 'S3','%CPR']].dropna(), on='Date', how='left')

In [180]:
fully_merged[pd.isna(fully_merged['PP'] )]

,Datetime,Open,High,Low,Close,Volume,Date,VWMA_10,Supertrend,PP,TC,BC,R1,R2,R3,S1,S2,S3,%CPR
0,2020-07-03 10:05:00,66.70,66.90,66.65,66.75,24634,2020-07-03,66.998609,64.535912,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-07-03 10:10:00,66.70,66.70,66.40,66.65,39420,2020-07-03,66.830156,64.535912,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-07-03 10:15:00,66.65,66.65,66.20,66.30,35586,2020-07-03,66.677180,64.535912,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-07-03 10:20:00,66.40,66.70,66.35,66.45,48965,2020-07-03,66.600970,64.535912,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-07-03 10:25:00,66.50,66.60,66.35,66.50,19003,2020-07-03,66.584594,64.596585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62409,2023-11-12 18:50:00,175.65,176.00,175.55,175.80,31630,2023-11-12,175.617814,173.891797,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62410,2023-11-12 18:55:00,175.95,175.95,175.70,175.80,17317,2023-11-12,175.646435,173.891797,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62411,2023-11-12 19:00:00,175.80,175.90,175.30,175.40,41044,2023-11-12,175.650197,173.891797,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62412,2023-11-12 19:05:00,175.40,175.60,175.25,175.55,48507,2023-11-12,175.797250,173.891797,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [181]:
fully_merged = fully_merged.fillna(method='ffill').dropna()

/var/folders/ds/wkxfdzns0q9b361mzl9c0_840000gn/T/ipykernel_1256/4200669879.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  fully_merged = fully_merged.fillna(method='ffill').dropna()


In [182]:
fully_merged.set_index('Datetime',inplace=True)

In [183]:
fully_merged

,Open,High,Low,Close,Volume,Date,VWMA_10,Supertrend,PP,TC,BC,R1,R2,R3,S1,S2,S3,%CPR
Datetime,,,,,,,,,,,,,,,,,,
2020-07-06 09:15:00,66.00,66.35,65.75,66.25,117808,2020-07-06,65.752785,66.426092,66.233333,67.116667,65.666667,67.366667,69.133333,70.266667,64.466667,63.333333,61.566667,0.021892
2020-07-06 09:20:00,66.15,66.40,66.00,66.35,66875,2020-07-06,65.819592,66.426092,66.233333,67.116667,65.666667,67.366667,69.133333,70.266667,64.466667,63.333333,61.566667,0.021892
2020-07-06 09:25:00,66.35,66.70,66.20,66.35,57628,2020-07-06,65.877779,66.426092,66.233333,67.116667,65.666667,67.366667,69.133333,70.266667,64.466667,63.333333,61.566667,0.021892
2020-07-06 09:30:00,66.30,66.60,66.30,66.45,35460,2020-07-06,65.917392,65.119099,66.233333,67.116667,65.666667,67.366667,69.133333,70.266667,64.466667,63.333333,61.566667,0.021892
2020-07-06 09:35:00,66.50,66.60,66.35,66.35,55546,2020-07-06,65.966544,65.177197,66.233333,67.116667,65.666667,67.366667,69.133333,70.266667,64.466667,63.333333,61.566667,0.021892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-09 15:05:00,212.53,212.55,212.37,212.55,14511,2024-08-09,212.595716,211.608214,210.983333,212.566667,209.791667,213.366667,216.533333,218.916667,207.816667,205.433333,202.266667,0.013153
2024-08-09 15:10:00,212.64,212.75,211.80,211.92,82129,2024-08-09,212.362561,211.608214,210.983333,212.566667,209.791667,213.366667,216.533333,218.916667,207.816667,205.433333,202.266667,0.013153
2024-08-09 15:15:00,211.83,211.89,211.00,211.53,61012,2024-08-09,212.174072,213.549858,210.983333,212.566667,209.791667,213.366667,216.533333,218.916667,207.816667,205.433333,202.266667,0.013153


In [184]:
fully_merged['Signal'] = np.where(
    (fully_merged['Close'] > fully_merged['R1']) & 
    (fully_merged['VWMA_10'] > fully_merged['R1']) & 
    (fully_merged['VWMA_10'].shift(1) < fully_merged['R1'].shift(1)), 
    1, 
   (np.where( (fully_merged['Close'] < fully_merged['S1']) & 
    (fully_merged['VWMA_10'] < fully_merged['S1']) & 
    (fully_merged['VWMA_10'].shift(1) > fully_merged['S1'].shift(1)), 
    -1, 0))
)

In [185]:
fully_merged.to_csv('fullymerged.csv')

In [186]:
# Define exit conditions based on the user's criteria
fully_merged['Exit_Condition'] = np.where(
    
    ((fully_merged['Close'] < fully_merged['TC']) | 
     (fully_merged['VWMA_10'] < fully_merged['PP'])),
    1, 
    np.where(
        ((fully_merged['Close'] > fully_merged['TC']) | 
         (fully_merged['VWMA_10'] > fully_merged['PP'])),
        -1, 0)
)


In [187]:
# Define the conditions for generating signals
fully_merged['Signal'] = np.where(
    (fully_merged['Close'] > fully_merged['R1']) & 
    (fully_merged['VWMA_10'] > fully_merged['R1']) & 
    (fully_merged['VWMA_10'].shift(1) < fully_merged['R1'].shift(1)), 
    1, 
   (np.where( (fully_merged['Close'] < fully_merged['S1']) & 
    (fully_merged['VWMA_10'] < fully_merged['S1']) & 
    (fully_merged['VWMA_10'].shift(1) > fully_merged['S1'].shift(1)), 
    -1, 0))
)


In [189]:
fully_merged['%CPR_condition'] = fully_merged['%CPR'] < 0.01


In [190]:
# Define target and stop-loss percentages
target_pct = 0.02  # 2% target
stop_loss_pct = 0.01  # 1% stop loss

# Initialize variables to store entry prices
buy_entry_price = None
short_entry_price = None



In [191]:
# Buy Entry Condition
fully_merged['Buy_Entry'] = (
    (fully_merged['Close'] > fully_merged['R1']) &
    (fully_merged['VWMA_10'].shift(1) <= fully_merged['R1'].shift(1)) & 
    (fully_merged['VWMA_10'] > fully_merged['R1']) &
    fully_merged['%CPR_condition']
)

# Short Entry Condition
fully_merged['Short_Entry'] = (
    (fully_merged['Close'] < fully_merged['S1']) &
    (fully_merged['VWMA_10'].shift(1) >= fully_merged['S1'].shift(1)) & 
    (fully_merged['VWMA_10'] < fully_merged['S1']) &
    fully_merged['%CPR_condition']
)


In [192]:
# Calculate entry prices and set target/stop-loss based on entry prices
for i in range(len(fully_merged)):
    if fully_merged['Buy_Entry'].iloc[i]:
        buy_entry_price = fully_merged['Close'].iloc[i]
        fully_merged.loc[fully_merged.index[i], 'Buy_Target'] = buy_entry_price * (1 + target_pct)
        fully_merged.loc[fully_merged.index[i], 'Buy_Stop_Loss'] = buy_entry_price * (1 - stop_loss_pct)
    elif fully_merged['Short_Entry'].iloc[i]:
        short_entry_price = fully_merged['Close'].iloc[i]
        fully_merged.loc[fully_merged.index[i], 'Short_Target'] = short_entry_price * (1 - target_pct)
        fully_merged.loc[fully_merged.index[i], 'Short_Stop_Loss'] = short_entry_price * (1 + stop_loss_pct)
    else:
        # Carry forward the last calculated target and stop-loss
        if buy_entry_price is not None:
            fully_merged.loc[fully_merged.index[i], 'Buy_Target'] = fully_merged['Buy_Target'].iloc[i-1]
            fully_merged.loc[fully_merged.index[i], 'Buy_Stop_Loss'] = fully_merged['Buy_Stop_Loss'].iloc[i-1]
        if short_entry_price is not None:
            fully_merged.loc[fully_merged.index[i], 'Short_Target'] = fully_merged['Short_Target'].iloc[i-1]
            fully_merged.loc[fully_merged.index[i], 'Short_Stop_Loss'] = fully_merged['Short_Stop_Loss'].iloc[i-1]


In [193]:

# Buy Exit Condition
fully_merged['Buy_Exit'] = (
    (fully_merged['Close'] < fully_merged['TC']) |
    (fully_merged['VWMA_10'].shift(1) >= fully_merged['PP'].shift(1)) & 
    (fully_merged['VWMA_10'] < fully_merged['PP'])|
    (fully_merged['Close'] >= fully_merged['Buy_Target']) |  # Target Condition
    (fully_merged['Close'] <= fully_merged['Buy_Stop_Loss'])  # Stop-Loss Condition
)

# Short Exit Condition
fully_merged['Short_Exit'] = (
    (fully_merged['Close'] > fully_merged['BC']) |
    (fully_merged['VWMA_10'].shift(1) <= fully_merged['PP'].shift(1)) & 
    (fully_merged['VWMA_10'] > fully_merged['PP'])|
    (fully_merged['Close'] <= fully_merged['Short_Target']) |  # Target Condition
    (fully_merged['Close'] >= fully_merged['Short_Stop_Loss'])  # Stop-Loss Condition
)


In [194]:
fully_merged

,Open,High,Low,Close,Volume,Date,VWMA_10,Supertrend,PP,TC,...,Exit_Condition,%CPR_condition,Buy_Entry,Short_Entry,Buy_Target,Buy_Stop_Loss,Short_Target,Short_Stop_Loss,Buy_Exit,Short_Exit
Datetime,,,,,,,,,,,,,,,,,,,,,
2020-07-06 09:15:00,66.00,66.35,65.75,66.25,117808,2020-07-06,65.752785,66.426092,66.233333,67.116667,...,1,False,False,False,NaN,NaN,NaN,NaN,True,True
2020-07-06 09:20:00,66.15,66.40,66.00,66.35,66875,2020-07-06,65.819592,66.426092,66.233333,67.116667,...,1,False,False,False,NaN,NaN,NaN,NaN,True,True
2020-07-06 09:25:00,66.35,66.70,66.20,66.35,57628,2020-07-06,65.877779,66.426092,66.233333,67.116667,...,1,False,False,False,NaN,NaN,NaN,NaN,True,True
2020-07-06 09:30:00,66.30,66.60,66.30,66.45,35460,2020-07-06,65.917392,65.119099,66.233333,67.116667,...,1,False,False,False,NaN,NaN,NaN,NaN,True,True
2020-07-06 09:35:00,66.50,66.60,66.35,66.35,55546,2020-07-06,65.966544,65.177197,66.233333,67.116667,...,1,False,False,False,NaN,NaN,NaN,NaN,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-09 15:05:00,212.53,212.55,212.37,212.55,14511,2024-08-09,212.595716,211.608214,210.983333,212.566667,...,1,False,False,False,229.704,222.948,NaN,NaN,True,True
2024-08-09 15:10:00,212.64,212.75,211.80,211.92,82129,2024-08-09,212.362561,211.608214,210.983333,212.566667,...,1,False,False,False,229.704,222.948,NaN,NaN,True,True
2024-08-09 15:15:00,211.83,211.89,211.00,211.53,61012,2024-08-09,212.174072,213.549858,210.983333,212.566667,...,1,False,False,False,229.704,222.948,NaN,NaN,True,True


In [195]:
# Initialize variables
initial_cash = 100000  # Starting with 100,000 units of currency
cash = initial_cash
position = 0  # Current position: 0 means no position, 1 means long, -1 means short
entry_price = 0
trade_history = []
risk_per_trade = 0.01  # Risk 1% of current cash balance per trade
target_pct = 0.02  # 2% target
stop_loss_pct = 0.01  # 1% stop loss


In [196]:
import pandas as pd
import numpy as np

# Assume fully_merged is your DataFrame with all the calculated conditions

# Define target and stop-loss percentages
target_pct = 0.02  # 2% target
stop_loss_pct = 0.01  # 1% stop loss
trade_charge = 40  # 40 units charge per trade

# Initialize variables
initial_cash = 100000  # Starting with 100,000 units of currency
cash = initial_cash
position = 0  # Current position: 0 means no position, 1 means long, -1 means short
buy_entry_price = None
short_entry_price = None
trade_history = []

# Iterate through the DataFrame
for i in range(len(fully_merged)):
    # Set up target and stop-loss for Buy Entry
    if fully_merged['Buy_Entry'].iloc[i] and position == 0:
        buy_entry_price = fully_merged['Close'].iloc[i]
        target_price = buy_entry_price * (1 + target_pct)
        stop_loss_price = buy_entry_price * (1 - stop_loss_pct)
        position = 1
        position_size = cash * 0.01 / (buy_entry_price - stop_loss_price)
        cash -= buy_entry_price * position_size + trade_charge  # Include trade charge
        trade_history.append({
            'Datetime': fully_merged.index[i], 
            'Type': 'Buy', 
            'Price': buy_entry_price, 
            'Size': position_size,
            'Target': target_price,
            'Stop_Loss': stop_loss_price
        })

    # Set up target and stop-loss for Short Entry
    elif fully_merged['Short_Entry'].iloc[i] and position == 0:
        short_entry_price = fully_merged['Close'].iloc[i]
        target_price = short_entry_price * (1 - target_pct)
        stop_loss_price = short_entry_price * (1 + stop_loss_pct)
        position = -1
        position_size = cash * 0.01 / (stop_loss_price - short_entry_price)
        cash += short_entry_price * position_size - trade_charge  # Include trade charge
        trade_history.append({
            'Datetime': fully_merged.index[i], 
            'Type': 'Short', 
            'Price': short_entry_price, 
            'Size': position_size,
            'Target': target_price,
            'Stop_Loss': stop_loss_price
        })

    # Exit conditions for Buy position
    if position == 1:
        if fully_merged['Close'].iloc[i] >= target_price:  # Target hit
            exit_price = target_price
            cash += exit_price * position_size - trade_charge  # Include trade charge
            position = 0
            profit = (exit_price - buy_entry_price) * position_size - trade_charge
            trade_history.append({
                'Datetime': fully_merged.index[i], 
                'Type': 'Sell (Target)', 
                'Price': exit_price, 
                'Profit': profit
            })
        elif fully_merged['Close'].iloc[i] <= stop_loss_price:  # Stop-loss hit
            exit_price = stop_loss_price
            cash += exit_price * position_size - trade_charge  # Include trade charge
            position = 0
            profit = (exit_price - buy_entry_price) * position_size - trade_charge
            trade_history.append({
                'Datetime': fully_merged.index[i], 
                'Type': 'Sell (Stop Loss)', 
                'Price': exit_price, 
                'Profit': profit
            })
        elif fully_merged['Buy_Exit'].iloc[i]:  # Regular exit condition
            exit_price = fully_merged['Close'].iloc[i]
            cash += exit_price * position_size - trade_charge  # Include trade charge
            position = 0
            profit = (exit_price - buy_entry_price) * position_size - trade_charge
            trade_history.append({
                'Datetime': fully_merged.index[i], 
                'Type': 'Sell (Exit)', 
                'Price': exit_price, 
                'Profit': profit
            })

    # Exit conditions for Short position
    if position == -1:
        if fully_merged['Close'].iloc[i] <= target_price:  # Target hit
            exit_price = target_price
            cash -= exit_price * position_size + trade_charge  # Include trade charge
            position = 0
            profit = (short_entry_price - exit_price) * position_size - trade_charge
            trade_history.append({
                'Datetime': fully_merged.index[i], 
                'Type': 'Cover (Target)', 
                'Price': exit_price, 
                'Profit': profit
            })
        elif fully_merged['Close'].iloc[i] >= stop_loss_price:  # Stop-loss hit
            exit_price = stop_loss_price
            cash -= exit_price * position_size + trade_charge  # Include trade charge
            position = 0
            profit = (short_entry_price - exit_price) * position_size - trade_charge
            trade_history.append({
                'Datetime': fully_merged.index[i], 
                'Type': 'Cover (Stop Loss)', 
                'Price': exit_price, 
                'Profit': profit
            })
        elif fully_merged['Short_Exit'].iloc[i]:  # Regular exit condition
            exit_price = fully_merged['Close'].iloc[i]
            cash -= exit_price * position_size + trade_charge  # Include trade charge
            position = 0
            profit = (short_entry_price - exit_price) * position_size - trade_charge
            trade_history.append({
                'Datetime': fully_merged.index[i], 
                'Type': 'Cover (Exit)', 
                'Price': exit_price, 
                'Profit': profit
            })

    # Check for end of day (intraday close)
    if fully_merged.index[i].time() == pd.to_datetime("15:15").time() and position != 0:
        exit_price = fully_merged['Close'].iloc[i]
        if position == 1:  # If in a long position
            cash += exit_price * position_size - trade_charge  # Include trade charge
            trade_history.append({
                'Datetime': fully_merged.index[i], 
                'Type': 'Sell (End of Day)', 
                'Price': exit_price, 
                'Profit': (exit_price - buy_entry_price) * position_size - trade_charge
            })
        elif position == -1:  # If in a short position
            cash -= exit_price * position_size + trade_charge  # Include trade charge
            trade_history.append({
                'Datetime': fully_merged.index[i], 
                'Type': 'Cover (End of Day)', 
                'Price': exit_price, 
                'Profit': (short_entry_price - exit_price) * position_size - trade_charge
            })
        position = 0  # Reset position to zero

# Final Portfolio Value
final_value = cash

# Convert trade history to DataFrame
trade_history_df = pd.DataFrame(trade_history)

# Calculate KPIs
total_trades = len(trade_history_df) // 2  # Assuming Buy/Sell pairs or Short/Cover pairs
winning_trades = trade_history_df[trade_history_df['Profit'] > 0]
losing_trades = trade_history_df[trade_history_df['Profit'] <= 0]

total_profit = trade_history_df['Profit'].sum()
win_rate = len(winning_trades) / total_trades if total_trades > 0 else 0
average_profit = trade_history_df['Profit'].mean()
profit_factor = winning_trades['Profit'].sum() / abs(losing_trades['Profit'].sum()) if len(losing_trades) > 0 else np.inf
max_drawdown = (trade_history_df['Profit'].cumsum().cummax() - trade_history_df['Profit'].cumsum()).max()
sharpe_ratio = trade_history_df['Profit'].mean() / trade_history_df['Profit'].std() * np.sqrt(252) if trade_history_df['Profit'].std() > 0 else np.nan
sortino_ratio = trade_history_df['Profit'].mean() / trade_history_df[trade_history_df['Profit'] < 0]['Profit'].std() * np.sqrt(252) if trade_history_df[trade_history_df['Profit'] < 0]['Profit'].std() > 0 else np.nan

# Output Results
print(f"Initial Cash: {initial_cash}")
print(f"Final Portfolio Value: {final_value}")
print(f"Total Profit: {total_profit}")
print(f"Return Percentage: {((final_value - initial_cash) / initial_cash) * 100:.2f}%")
print(f"Number of Trades: {total_trades}")
print(f"Win Rate: {win_rate:.2f}")
print(f"Average Profit/Loss per Trade: {average_profit:.2f}")
print(f"Profit Factor: {profit_factor:.2f}")
print(f"Max Drawdown: {max_drawdown:.2f}")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
print(f"Sortino Ratio: {sortino_ratio:.2f}")



Initial Cash: 100000
Final Portfolio Value: 105149.5641701834
Total Profit: 12109.5641701835
Return Percentage: 5.15%
Number of Trades: 174
Win Rate: 0.42
Average Profit/Loss per Trade: 69.60
Profit Factor: 1.18
Max Drawdown: 14028.35
Sharpe Ratio: 1.06
Sortino Ratio: 2.84


,Datetime,Type,Price,Size,Target,Stop_Loss,Profit
0,2020-08-19 09:45:00,Buy,60.8000,1644.736842,62.016,60.1920,NaN
1,2020-08-19 10:30:00,Sell (Target),62.0160,NaN,NaN,NaN,1960.000000
2,2020-10-05 09:55:00,Buy,65.1000,1565.591398,66.402,64.4490,NaN
3,2020-10-05 11:25:00,Sell (Stop Loss),64.4490,NaN,NaN,NaN,-1059.200000
4,2020-10-05 15:00:00,Buy,64.0500,1574.095238,65.331,63.4095,NaN
...,...,...,...,...,...,...,...
343,2024-07-25 10:55:00,Cover (Stop Loss),214.9684,NaN,NaN,NaN,-1076.286365
344,2024-07-25 15:20:00,Buy,219.0000,468.092923,223.380,216.8100,NaN
345,2024-07-26 15:05:00,Sell (Target),223.3800,NaN,NaN,NaN,2010.247002
346,2024-07-31 09:40:00,Buy,225.2000,463.954694,229.704,222.9480,NaN
